In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
plt.rcParams['figure.figsize'] = 10,10
from utils import abline, BacktrackingLineSearch
import pdb

Logistic regression is a discriminant approach, i.e., we are modeling the posterior probabilities $\pi(x)$ of the
labels directly using the logistic function, s.t.
$$\pi(x)=P(y=1/x)=\frac{1}{1+exp(-\theta^Tx)}$$
Note As in the lecture we suppress the intercept in notation, i.e. $\theta^Tx:=\theta_0+\theta^Tx$. This can be easily implemented as :

In [ ]:
def logistic(X,theta):
    return 1/(1+np.exp(-X.dot(theta)))

And we can observe that logistic regression “squashes” the estimated linear scores $\theta^Tx$ to [0, 1]:

In [ ]:
x = np.linspace(-6,6,100)
theta = 0.8
sns.lineplot(x,logistic(x,theta))
plt.xlabel(r"$\theta^Tx$");
plt.ylabel(r"$\pi(x)$");

From the definition of accuracy and the encoding of the labels to {−1, 1}, one can derive the Bernoulli loss (see lecture slides):
$$L(y,f(x))=log(1+exp(-yf(x)))$$
For logistic regression, we have $f(x) = \theta^Tx$. This is implemented and plotted as follows :

In [ ]:
def loss_bernoulli(theta,y,X):
#     pdb.set_trace()
    return np.log(1+np.exp(-y*X.dot(theta).reshape(-1,1))).squeeze()

We can plot the loss for y = 1 and y = −1:

In [ ]:
X = np.vstack([np.ones_like(x),x]).T
theta = np.array([0,1])
sns.lineplot(x,loss_bernoulli(y=1,X=X,theta=theta))
sns.lineplot(x,loss_bernoulli(y=-1,X=X,theta=theta))
plt.legend(['+1','-1'])
plt.xlabel('f(x)')
plt.ylabel('L(y,f(x))');

This means that, in order to minimize the loss, we should predict y = 1 if
$$\theta^Tx \ge 0 \Leftrightarrow \pi(x) = P(y=1/x) = \frac{1}{1+exp(-\theta^Tx)} \ge 0.5$$

In [ ]:
def classify_logreg(X,theta, threshold=0.5):
    prob = logistic(X,theta)
    return prob>threshold

With the help of the loss function, the emperical risk then becomes:
$$R_{emp}=\frac{1}{n}\sum_{i=1}^n L(y^{(i)},f(x^{(i)}))$$

In [ ]:
def risk_bernoulli(theta,y,X):
    return np.mean(loss_bernoulli(theta,y,X))

### Logistic regression - example
To investigate how the parameter $\theta$ of the logistic regression can be estimated, we create some artificial data:

In [ ]:
n=10000
p=2
slope = 3.0
np.random.seed(42)

X = np.random.uniform(-5,5,n*p).reshape(n,p)
theta = np.array([slope,-1])

y = (2*np.random.binomial(1,logistic(X,theta))-1).reshape(-1,1)

Then we create a logistic regression classifier from sklearn and plot the predictions after fitting the data

In [ ]:
clf = LogisticRegression(random_state=0).fit(X, y.ravel())
y_pred = clf.predict(X)

slope_pred,intercept_pred = clf.coef_[0]
print(slope_pred,intercept_pred)
abline(slope_pred, intercept_pred)
sns.scatterplot(X[:,0],X[:,1],hue=y.squeeze());

We notice that our data is linearly separated as one would expect, since our decision boundary is defined by $\theta^Tx=0$ which defines a hyperplane.

Furthermore, we note that $\theta$ represents a vector normal to this plane and is pointing to the “1”-side. But
doesn’t that mean that for any positive $\lambda \in \mathbb{R^+}$, a rescaled coefficient vector $\lambda\theta$, which defines the same
“direction” of decision boundary as $\theta$, would separate the data equally well. . . ?

Check the classification and the empirical risk for $\lambda \in \{0.5, 2, 3\}$. Can you figure out what determines the optimal length of $\theta$?

To gain further understanding of this behavior, let’s look at a data scenario where we encounter so-called **complete separation**, i.e., a situation in which the data can be
classified without error by our classification method. We can simply use the predictions from our model as if they were the observed labels to create such a dataset:

In [ ]:
y_compl_sep = clf.predict(X)
thetas = np.array([0.5,1.0,2.0,10.0])*theta.reshape(-1,1)

In [ ]:
thetas

Remember the Bernoulli loss was defined as : $L(y,f(x))=log(1+exp(-y\theta^Tx))$

In the case of complete separation, since every observation is classified correctly, hence $\theta^Tx>0$ for y=1 and $\theta^Tx<0$ for y=-1. Then the bernoulli loss becomes $L(y,f(x))=log(1+exp(-|\theta^Tx|))$.

This means the empirical risk is monotonously decreasing as the length of theta increases, which means that we never converge on a finite solution by minimizing the empirical risk. In this situation, additional artificial constraints – such as regularization – can be introduced to find a solution. In paractice, complete separation can happen fairly easily in datasets with a small number of observations compared to the number of available features.

Since we don’t need to deal with complete separation in our example, we can simply apply the gradient_descent_opt_stepsize function of the **code demo to the linear model** directly to the empirical risk we derived from the Bernoulli loss. This type of gradient descent mimization can be applied to any risk for which we can figure out the gradient, in this case this is

$$\frac{\partial R_{emp}(f)}{\partial\theta} = \sum_{i=1}^n \frac{-y^{(i)}}{1+exp(y^{(i)}\theta^Tx^{(i)})}x^{(i)} $$

Which can be implemented as :

In [ ]:
def gradient_bernoulli(theta,y,X):
    loss_i = -y*X/(1+np.exp(-y*X.dot(theta).reshape(-1,1)))
    return np.sum(loss_i,axis=0)

Now we can apply the gradient descent method to find a numerical solution:

In [ ]:
def gradient_descent_opt_stepsize(y,X,theta,
                                  loss_fn=risk_bernoulli,
                                  grad_fn=gradient_bernoulli,
                                  max_iter=100,
                                  threshold=1e-8):

    loss_storage=[]
    theta_storage=[]
    lr_storage=[]
    
    loss_storage.append(loss_fn(theta,y,X))
    theta_storage.append(theta)
    
    for i in range(max_iter):
        lr_opt = BacktrackingLineSearch(loss_fn,grad_fn,theta,-grad_fn(theta,y,X),args=(y,X))
#         print(i,lr_opt)
        theta = theta - lr_opt * grad_fn(theta,y,X)
        loss = loss_fn(theta,y,X)
        
        if (i>1)&(np.sqrt(np.sum((theta_storage[-1]-theta)**2))<threshold):
            print(f'threshold reached. Breaking at iteration : {i}')
            break
        loss_storage.append(loss)
        theta_storage.append(theta)
        lr_storage.append(lr_opt)
        
    return loss_storage,theta_storage,lr_storage

In [ ]:
theta=thetas[:,0]
print(theta)
theta_init=np.array([-1,-1.5])
y = (2*np.random.binomial(1,logistic(X,theta))-1).reshape(-1,1)

In [ ]:
out=gradient_descent_opt_stepsize(y,X,theta_init)

In [ ]:
out[0]

In [ ]:
out[1]

In [ ]:
out[2]

In [ ]:
theta